Jamison Hunter

July 6, 2023

# Yelp API Calls

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

This demonstrates that the client ID and API key are accessible. 

In [2]:
# Load API Credentials
with open('/Users/Jamison/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Indianapolis, IN'
TERM = 'Pizza'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_indianapolis_pizza.json"
JSON_FILE

'Data/results_in_progress_indianapolis_pizza.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_indianapolis_pizza.json already exists.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 440 previous results found.


In [7]:
# Quick Test Query
results = yelp_api.search_query(location=LOCATION,
                                       term=TERM)
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [8]:
# writing Indianapolis pizza location data onto a json file 
with open(JSON_FILE,'w') as f:
    json.dump(results, f)
# Load saved json file back to dictionary
with open(JSON_FILE) as f:
    loaded = json.load(f)

In [9]:
# checking to make sure json file dictionary keys are readable
loaded.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
len(loaded["businesses"])

20

In [11]:
df_businesses = pd.DataFrame(loaded['businesses'])
df_businesses.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wD0OY5AYINSCyCflHgd7Ag,futuro-indianapolis,Futuro,https://s3-media4.fl.yelpcdn.com/bphoto/h_sfLx...,False,https://www.yelp.com/biz/futuro-indianapolis?a...,192,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 39.7675986, 'longitude': -86.1406...","[delivery, pickup]",$$,"{'address1': '19 Cruse St', 'address2': '', 'a...",+13173604725,(317) 360-4725,4301.650116
1,C3Pf_LNFt1GeiRaYBRQhJg,pizza-palace-california-style-indianapolis,Pizza Palace California Style,https://s3-media2.fl.yelpcdn.com/bphoto/mdwRtt...,False,https://www.yelp.com/biz/pizza-palace-californ...,20,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 39.897913, 'longitude': -86.217173}","[delivery, pickup]",NaN,"{'address1': '7940 Michigan Rd', 'address2': '...",+13179749041,(317) 974-9041,12210.440284
2,yI-ahfduU45_bZa3ZgEx4w,jockamo-upper-crust-pizza-indianapolis,Jockamo Upper Crust Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/lUW4ck...,False,https://www.yelp.com/biz/jockamo-upper-crust-p...,675,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.771, 'longitude': -86.07055}",[delivery],$$,"{'address1': '5646 E Washington St', 'address2...",+13173566612,(317) 356-6612,6981.678615
3,pDghOix5B17eZTtjDsQ4_A,king-dough-indy-indianapolis,King Dough Indy,https://s3-media3.fl.yelpcdn.com/bphoto/Xl5u1c...,False,https://www.yelp.com/biz/king-dough-indy-india...,440,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.7739378, 'longitude': -86.1374...","[delivery, pickup]",$$,"{'address1': '452 N Highland Ave', 'address2':...",+13176027960,(317) 602-7960,3593.069780
4,Y6bKMtyK02vhA5doR_z7qA,greek-s-pizzeria-indianapolis-indianapolis,Greek’s Pizzeria- Indianapolis,https://s3-media3.fl.yelpcdn.com/bphoto/4Ni2fc...,False,https://www.yelp.com/biz/greek-s-pizzeria-indi...,96,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.78867, 'longitude': -86.13631}",[delivery],NaN,"{'address1': '1601 Columbia Ave', 'address2': ...",+13176026833,(317) 602-6833,1962.026949


In [12]:
df_businesses[['categories','coordinates','location']].head()

,categories,coordinates,location
0,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 39.7675986, 'longitude': -86.1406...","{'address1': '19 Cruse St', 'address2': '', 'a..."
1,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 39.897913, 'longitude': -86.217173}","{'address1': '7940 Michigan Rd', 'address2': '..."
2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 39.771, 'longitude': -86.07055}","{'address1': '5646 E Washington St', 'address2..."
3,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 39.7739378, 'longitude': -86.1374...","{'address1': '452 N Highland Ave', 'address2':..."
4,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 39.78867, 'longitude': -86.13631}","{'address1': '1601 Columbia Ave', 'address2': ..."


In [31]:
## How many results total?
total_results = results['total']
total_results

1500

In [32]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [33]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

53

In [34]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [35]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [38]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:

        ## Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [39]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_indianapolis_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


75

In [40]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/75 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [41]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wD0OY5AYINSCyCflHgd7Ag,futuro-indianapolis,Futuro,https://s3-media4.fl.yelpcdn.com/bphoto/h_sfLx...,False,https://www.yelp.com/biz/futuro-indianapolis?a...,192,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 39.7675986, 'longitude': -86.1406...","[pickup, delivery]",$$,"{'address1': '19 Cruse St', 'address2': '', 'a...",+13173604725,(317) 360-4725,4301.650116
1,C3Pf_LNFt1GeiRaYBRQhJg,pizza-palace-california-style-indianapolis,Pizza Palace California Style,https://s3-media2.fl.yelpcdn.com/bphoto/mdwRtt...,False,https://www.yelp.com/biz/pizza-palace-californ...,20,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 39.897913, 'longitude': -86.217173}","[pickup, delivery]",NaN,"{'address1': '7940 Michigan Rd', 'address2': '...",+13179749041,(317) 974-9041,12210.440284
2,yI-ahfduU45_bZa3ZgEx4w,jockamo-upper-crust-pizza-indianapolis,Jockamo Upper Crust Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/lUW4ck...,False,https://www.yelp.com/biz/jockamo-upper-crust-p...,675,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.771, 'longitude': -86.07055}",[delivery],$$,"{'address1': '5646 E Washington St', 'address2...",+13173566612,(317) 356-6612,6981.678615
3,pDghOix5B17eZTtjDsQ4_A,king-dough-indy-indianapolis,King Dough Indy,https://s3-media3.fl.yelpcdn.com/bphoto/Xl5u1c...,False,https://www.yelp.com/biz/king-dough-indy-india...,440,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.7739378, 'longitude': -86.1374...","[pickup, delivery]",$$,"{'address1': '452 N Highland Ave', 'address2':...",+13176027960,(317) 602-7960,3593.069780
4,Y6bKMtyK02vhA5doR_z7qA,greek-s-pizzeria-indianapolis-indianapolis,Greek’s Pizzeria- Indianapolis,https://s3-media3.fl.yelpcdn.com/bphoto/4Ni2fc...,False,https://www.yelp.com/biz/greek-s-pizzeria-indi...,96,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.78867, 'longitude': -86.13631}",[delivery],NaN,"{'address1': '1601 Columbia Ave', 'address2': ...",+13176026833,(317) 602-6833,1962.026949


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,DzLWIdOgeEdCodM27X6eGg,el-valle-minimarket-indianapolis,El Valle Minimarket,https://s3-media2.fl.yelpcdn.com/bphoto/PCKddA...,False,https://www.yelp.com/biz/el-valle-minimarket-i...,14,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 39.6586, 'longitude': -86.1254}",[delivery],NaN,"{'address1': '7482 Madison Ave', 'address2': N...",+13177569187,(317) 756-9187,16504.139557
996,YbkKJ1AqDnG7rdMkDXktpQ,little-caesars-avon-2,Little Caesars,https://s3-media2.fl.yelpcdn.com/bphoto/2lc5Ap...,False,https://www.yelp.com/biz/little-caesars-avon-2...,3,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.7638349, 'longitude': -86.3272...",[],$,"{'address1': '10976 US Hwy 36', 'address2': ''...",+13172099995,(317) 209-9995,16793.179898
997,MvWBAo164qmZGu_04b08vw,bamboo-house-indianapolis-2,Bamboo House,https://s3-media2.fl.yelpcdn.com/bphoto/ae9kjs...,False,https://www.yelp.com/biz/bamboo-house-indianap...,81,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 39.8494606, 'longitude': -86.1231...","[delivery, pickup]",$,"{'address1': '2330 E 53rd St', 'address2': '',...",+13172518080,(317) 251-8080,4981.904568
998,qJAWrsvNv74KYAoo6TAj0A,long-s-bakery-indianapolis,Long’s Bakery,https://s3-media1.fl.yelpcdn.com/bphoto/29xvrY...,False,https://www.yelp.com/biz/long-s-bakery-indiana...,366,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,"{'latitude': 39.664815837887524, 'longitude': ...",[],$,"{'address1': '2301 E Southport Rd', 'address2'...",+13177831442,(317) 783-1442,15790.119889
999,rMfI2VVaZoDgF-GYdvOw8A,wings-and-kabob-indianapolis-2,Wings & Kabob,https://s3-media1.fl.yelpcdn.com/bphoto/aoigS9...,False,https://www.yelp.com/biz/wings-and-kabob-india...,24,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,"{'latitude': 39.88333, 'longitude': -86.208414}","[delivery, pickup]",NaN,"{'address1': '7133 Michigan Rd', 'address2': N...",+13179815692,(317) 981-5692,10454.500558


In [42]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_indianapolis_pizza.csv.gz', compression='gzip',index=False)